In [1]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np

In [2]:
# importing csv that contains the stock ticker symbols and names of the S&P 500.
sandp_df = pd.read_csv('../Resources/SP_500.csv')
russell_2000_df = pd.read_csv('../Resources/russell_2000.csv')

In [3]:
# creating empty list.
ticker_symbols = []

# looping through above csv and appending stock ticker symbols to the empty list.
# for ticker in sandp_df['Ticker']:
#     ticker_symbols.append(ticker)
    
# looping through above csv and appending stock ticker symbols to the empty list.
for ticker in russell_2000_df['Ticker']:
    ticker_symbols.append(ticker)
    

# creating empty list.
names = []  

# looping through csv and appending names to the empty list.
# for name in sandp_df['Name']:
#     names.append(name)

# looping through csv and appending names to the empty list.
for name in russell_2000_df['Name']:
    names.append(name)

In [5]:
# creating/initializing an empty pandas dataframe.
dataframe2 = pd.DataFrame()

# creating loop that goes through the ticker_symbols and names lists created above.
for (ticker, name) in zip(ticker_symbols, names):
    
    # creating a for loop that goes through the last two entry's value of the json dictionaries from financialmodelingprep.    
    for i in range(-2,0):
        
        # setting up try and except statement to avoid lack of financialmodelingprep json data to ticker_symbol list.         
        try:
            
            # obtaining the json data for financial ratios from financialmodelingprep.          
            response = requests.get(f'https://financialmodelingprep.com/api/v3/financial-ratios/{ticker}').json()
            
            print(f'Processing:{ticker} {name}')
            
            # creating empty list named columns.             
            columns = []
            
            # appending 'symbol' to columns list.                
            columns.append('symbol')
            
            # appending 'name' to columns list.            
            columns.append('name')
        
            # appending 'date' to columns list.         
            columns.append('date')
            
            # creating empty list named data.            
            data = []
            
            # filtering the the financial ratios json dicionary data by years 2009 and 2010.            
            if response['ratios'][i]['date'].split('-')[0] == '2009' or response['ratios'][i]['date'].split('-')[0] == '2010':
                
                # creating another try except statement to avoid pitfalls.                
                try:
                    
                    # appending symbol to data list.
                    data.append(response['symbol'])
                   
                    # appending company name to data list.
                    data.append(name)
                    
                    # appending date to data list.
                    data.append(response['ratios'][i]['date'])
                    
                    # saving financial ratio dicionary to variable.
                    keys = response['ratios'][i]
                    
                    # looping through finanial ratio keys.                   
                    for key in keys:
                        
                        # avoiding date key.                       
                        if key != 'date':
                            
                            # looping through finanial ratio dictionary.                            
                            for x in keys[key]:
                                
                                #  appending financial ratio dict keys to columns list.                                
                                columns.append(x)
                                
                                # filtering financial ratio values that are blank.                 
                                if keys[key][x] == '':
                                    
                                    # appending 0 in the spot of nothing to the data list.                                     
                                    data.append(0)
                                
                                # continuing the filtering process, but with some values.         
                                else:
                                    
                                    # appending those financial ratio values to data list.                                 
                                    data.append(float(keys[key][x]))
                    
                    # creating a connection and obtaining financial growth statistics from financialmodelingprep.
                    response2 = requests.get(f'https://financialmodelingprep.com/api/v3/financial-statement-growth/{ticker}').json()
                    
                    # saving financial growth dict to variable.                  
                    keys2 = response2['growth'][i]
                    
                    # creating for loop to go through financial grow. 
                    for key in keys2:
                        
                        # avoiding the date key.                       
                        if key != 'date':
                            
                            # appending the financial growth keys to columns list.                            
                            columns.append(key)
                            
                            # filtering financial growth values that have nan values.
                            if keys2[key] == '':

                                # replacing nan values with 0.                                 
                                data.append(0)

                            # filtering financial growth values that are not nan.
                            else:

                                # appending those financial growth values to the data list.        
                                data.append(float(keys2[key]))

                    # creating a connection to get the company key metrics data from financialmodelingprep and putting in json form.
                    response3 = requests.get(f'https://financialmodelingprep.com/api/v3/company-key-metrics/{ticker}').json()
                    
                    # filtering market cap metric values that are nan.
                    if response3['metrics'][1]['Market Cap'] == '':
                        
                        # setting returns variable to 0 based on no market cap data.                    
                        returns = 0
                    
                    # filtering market cap based on having a value.
                    else:
                        
                        # using market cap value of 2009 and 2018/2019 to calculated percentage return over the time span.                        
                        returns = ((((float(response3['metrics'][0]['Market Cap']) + float(response3['metrics'][1]['Market Cap']))/2) - ((float(response3['metrics'][-2]['Market Cap']) + float(response3['metrics'][-1]['Market Cap']))/2))/((float(response3['metrics'][-2]['Market Cap']) + float(response3['metrics'][-1]['Market Cap']))/2))*100
                    
                    # creating a pandas dataframe based on the data and column lists developed through the above process.
                    dataframe = pd.DataFrame([data], columns=columns)
                    
                    # appending a market return on investment to newly created dataframe.
                    dataframe['% return on investment'] = [returns]
                    
                    # appending the dataframe to the empty dataframe created at the very top.
                    dataframe2 = dataframe2.append(dataframe)
                
                # finishing try except statement.
                except:
                    
                    # telling program to move on and ignore pitfall.                    
                    pass
            
            # finishing date(2009,2010) if then statement created near the bottom of the top.         
            else:
                
                 # telling program to move on and ignore pitfall.
                pass
        
        # finishing the other try except statement.        
        except:
            
             # telling program to move on and ignore pitfall.
            pass

# grouping completed dataframe by symbol and name and finding the average of the columns, because used multiple dates.
dataframe2 = dataframe2.groupby(['symbol', 'name']).mean()

# viewing dataframe.
dataframe2

Processing:AAN Aaron's, Inc.
Processing:AAN Aaron's, Inc.
Processing:AAOI Applied Optoelectronics, Inc.
Processing:AAOI Applied Optoelectronics, Inc.
Processing:AAON AAON, Inc.
Processing:AAON AAON, Inc.
Processing:AAT American Assets Trust, Inc.
Processing:AAT American Assets Trust, Inc.
Processing:AAWW Atlas Air Worldwide Holdings, Inc.
Processing:AAWW Atlas Air Worldwide Holdings, Inc.
Processing:AAXN Axon Enterprise, Inc.
Processing:AAXN Axon Enterprise, Inc.
Processing:ABCB Ameris Bancorp
Processing:ABCB Ameris Bancorp
Processing:ABEO Abeona Therapeutics, Inc.
Processing:ABEO Abeona Therapeutics, Inc.
Processing:ABG Asbury Automotive Group, Inc.
Processing:ABG Asbury Automotive Group, Inc.
Processing:ABM ABM Industries, Inc.
Processing:ABM ABM Industries, Inc.
Processing:ABTX Allegiance Bancshares, Inc. (Texas)
Processing:ABTX Allegiance Bancshares, Inc. (Texas)
Processing:AC Associated Capital Group, Inc.
Processing:AC Associated Capital Group, Inc.
Processing:ACA Arcosa, Inc.
Pr

Processing:AMSF AMERISAFE, Inc.
Processing:AMSF AMERISAFE, Inc.
Processing:AMSWA American Software, Inc.
Processing:AMSWA American Software, Inc.
Processing:AMTB Amerant Bancorp, Inc.
Processing:AMTB Amerant Bancorp, Inc.
Processing:AMWD American Woodmark Corp.
Processing:AMWD American Woodmark Corp.
Processing:ANAB AnaptysBio, Inc.
Processing:ANAB AnaptysBio, Inc.
Processing:ANDE The Andersons, Inc.
Processing:ANDE The Andersons, Inc.
Processing:ANF Abercrombie & Fitch Co.
Processing:ANF Abercrombie & Fitch Co.
Processing:ANGO AngioDynamics, Inc.
Processing:ANGO AngioDynamics, Inc.
Processing:ANH Anworth Mortgage Asset Corp.
Processing:ANH Anworth Mortgage Asset Corp.
Processing:ANIK Anika Therapeutics, Inc.
Processing:ANIK Anika Therapeutics, Inc.
Processing:ANIP ANI Pharmaceuticals, Inc.
Processing:ANIP ANI Pharmaceuticals, Inc.
Processing:AOBC American Outdoor Brands Corp.
Processing:AOBC American Outdoor Brands Corp.
Processing:AOSL Alpha & Omega Semiconductor Ltd.
Processing:AOSL

Processing:BCEI Bonanza Creek Energy, Inc.
Processing:BCEI Bonanza Creek Energy, Inc.
Processing:BCEL Atreca, Inc.
Processing:BCEL Atreca, Inc.
Processing:BCML BayCom Corp.
Processing:BCML BayCom Corp.
Processing:BCO The Brink's Co.
Processing:BCO The Brink's Co.
Processing:BCOR Blucora, Inc.
Processing:BCOR Blucora, Inc.
Processing:BCOV Brightcove, Inc.
Processing:BCOV Brightcove, Inc.
Processing:BCPC Balchem Corp.
Processing:BCPC Balchem Corp.
Processing:BCRX BioCryst Pharmaceuticals, Inc.
Processing:BCRX BioCryst Pharmaceuticals, Inc.
Processing:BDC Belden, Inc.
Processing:BDC Belden, Inc.
Processing:BDGE Bridge Bancorp, Inc.
Processing:BDGE Bridge Bancorp, Inc.
Processing:BDSI BioDelivery Sciences International, Inc.
Processing:BDSI BioDelivery Sciences International, Inc.
Processing:BE Bloom Energy Corp.
Processing:BE Bloom Energy Corp.
Processing:BEAT BioTelemetry, Inc.
Processing:BEAT BioTelemetry, Inc.
Processing:BECN Beacon Roofing Supply, Inc.
Processing:BECN Beacon Roofing S

Processing:CATC Cambridge Bancorp
Processing:CATM Cardtronics plc
Processing:CATM Cardtronics plc
Processing:CATO The Cato Corp.
Processing:CATO The Cato Corp.
Processing:CATS Catasys, Inc.
Processing:CATS Catasys, Inc.
Processing:CATY Cathay General Bancorp
Processing:CATY Cathay General Bancorp
Processing:CBAN Colony Bankcorp, Inc.
Processing:CBAN Colony Bankcorp, Inc.
Processing:CBAY CymaBay Therapeutics, Inc.
Processing:CBAY CymaBay Therapeutics, Inc.
Processing:CBB Cincinnati Bell, Inc.
Processing:CBB Cincinnati Bell, Inc.
Processing:CBL CBL & Associates Properties, Inc.
Processing:CBL CBL & Associates Properties, Inc.
Processing:CBLK Carbon Black, Inc. (Massachusetts)
Processing:CBLK Carbon Black, Inc. (Massachusetts)
Processing:CBM Cambrex Corporation
Processing:CBM Cambrex Corporation
Processing:CBMG Cellular Biomedicine Group, Inc.
Processing:CBMG Cellular Biomedicine Group, Inc.
Processing:CBNK Capital Bancorp, Inc. (Maryland)
Processing:CBNK Capital Bancorp, Inc. (Maryland)


Processing:CNOB ConnectOne Bancorp, Inc.
Processing:CNR Cornerstone Building Brands, Inc.
Processing:CNR Cornerstone Building Brands, Inc.
Processing:CNS Cohen & Steers, Inc. (New York)
Processing:CNS Cohen & Steers, Inc. (New York)
Processing:CNSL Consolidated Communications Holdings, Inc.
Processing:CNSL Consolidated Communications Holdings, Inc.
Processing:CNST Constellation Pharmaceuticals, Inc.
Processing:CNST Constellation Pharmaceuticals, Inc.
Processing:CNTY Century Casinos, Inc.
Processing:CNTY Century Casinos, Inc.
Processing:CNX CNX Resources Corp.
Processing:CNX CNX Resources Corp.
Processing:CNXN PC Connection, Inc.
Processing:CNXN PC Connection, Inc.
Processing:CODA Coda Octopus Group, Inc.
Processing:CODA Coda Octopus Group, Inc.
Processing:COHU Cohu, Inc.
Processing:COHU Cohu, Inc.
Processing:COKE Coca-Cola Consolidated, Inc.
Processing:COKE Coca-Cola Consolidated, Inc.
Processing:COLB Columbia Banking System, Inc.
Processing:COLB Columbia Banking System, Inc.
Processin

...

Processing:SALT Scorpio Bulkers, Inc.
Processing:SAM Boston Beer Co., Inc.
Processing:SAM Boston Beer Co., Inc.
Processing:SAMG Silvercrest Asset Management Group, Inc.
Processing:SAMG Silvercrest Asset Management Group, Inc.
Processing:SANM Sanmina Corp.
Processing:SANM Sanmina Corp.
Processing:SASR Sandy Spring Bancorp, Inc.
Processing:SASR Sandy Spring Bancorp, Inc.
Processing:SAVE Spirit Airlines, Inc.
Processing:SAVE Spirit Airlines, Inc.
Processing:SB Safe Bulkers, Inc.
Processing:SB Safe Bulkers, Inc.
Processing:SBBP Strongbridge Biopharma Plc
Processing:SBBP Strongbridge Biopharma Plc
Processing:SBBX SB One Bancorp
Processing:SBBX SB One Bancorp
Processing:SBCF Seacoast Banking Corp. of Florida
Processing:SBCF Seacoast Banking Corp. of Florida
Processing:SBH Sally Beauty Holdings, Inc.
Processing:SBH Sally Beauty Holdings, Inc.
Processing:SBOW SilverBow Resources, Inc.
Processing:SBOW SilverBow Resources, Inc.
Processing:SBRA Sabra Health Care REIT, Inc.
Processing:SBRA Sabra H

Processing:SRCI SRC Energy, Inc.
Processing:SRDX SurModics, Inc.
Processing:SRDX SurModics, Inc.
Processing:SRG Seritage Growth Properties
Processing:SRG Seritage Growth Properties
Processing:SRI Stoneridge, Inc.
Processing:SRI Stoneridge, Inc.
Processing:SRNE Sorrento Therapeutics, Inc.
Processing:SRNE Sorrento Therapeutics, Inc.
Processing:SRRK Scholar Rock Holding Corp.
Processing:SRRK Scholar Rock Holding Corp.
Processing:SRT Startek, Inc.
Processing:SRT Startek, Inc.
Processing:SSB South State Corp.
Processing:SSB South State Corp.
Processing:SSD Simpson Manufacturing Co., Inc.
Processing:SSD Simpson Manufacturing Co., Inc.
Processing:SSP The E.W. Scripps Co.
Processing:SSP The E.W. Scripps Co.
Processing:SSTI ShotSpotter, Inc.
Processing:SSTI ShotSpotter, Inc.
Processing:SSTK Shutterstock, Inc.
Processing:SSTK Shutterstock, Inc.
Processing:SSYS Stratasys Ltd.
Processing:SSYS Stratasys Ltd.
Processing:STAA STAAR Surgical Co.
Processing:STAA STAAR Surgical Co.
Processing:STAG STAG 

Processing:TPIC TPI Composites, Inc.
Processing:TPIC TPI Composites, Inc.
Processing:TPRE Third Point Reinsurance Ltd.
Processing:TPRE Third Point Reinsurance Ltd.
Processing:TPTX Turning Point Therapeutics, Inc.
Processing:TPTX Turning Point Therapeutics, Inc.
Processing:TR Tootsie Roll Industries, Inc.
Processing:TR Tootsie Roll Industries, Inc.
Processing:TRC Tejon Ranch Co.
Processing:TRC Tejon Ranch Co.
Processing:TREC Trecora Resources
Processing:TREC Trecora Resources
Processing:TREX Trex Co., Inc.
Processing:TREX Trex Co., Inc.
Processing:TRHC Tabula Rasa Healthcare, Inc.
Processing:TRHC Tabula Rasa Healthcare, Inc.
Processing:TRMK Trustmark Corp.
Processing:TRMK Trustmark Corp.
Processing:TRNO Terreno Realty Corp.
Processing:TRNO Terreno Realty Corp.
Processing:TRNS Transcat, Inc.
Processing:TRNS Transcat, Inc.
Processing:TROX Tronox Holdings Plc
Processing:TROX Tronox Holdings Plc
Processing:TRS TriMas Corp.
Processing:TRS TriMas Corp.
Processing:TRST TrustCo Bank Corp. NY
Pr

Processing:VPG Vishay Precision Group, Inc.
Processing:VPG Vishay Precision Group, Inc.
Processing:VRA Vera Bradley, Inc.
Processing:VRA Vera Bradley, Inc.
Processing:VRAY ViewRay, Inc.
Processing:VRAY ViewRay, Inc.
Processing:VRCA Verrica Pharmaceuticals, Inc.
Processing:VRCA Verrica Pharmaceuticals, Inc.
Processing:VREX Varex Imaging Corp.
Processing:VREX Varex Imaging Corp.
Processing:VRNS Varonis Systems, Inc.
Processing:VRNS Varonis Systems, Inc.
Processing:VRNT Verint Systems, Inc.
Processing:VRNT Verint Systems, Inc.
Processing:VRRM Verra Mobility Corp.
Processing:VRRM Verra Mobility Corp.
Processing:VRS Verso Corp.
Processing:VRS Verso Corp.
Processing:VRTS Virtus Investment Partners, Inc.
Processing:VRTS Virtus Investment Partners, Inc.
Processing:VRTU Virtusa Corp.
Processing:VRTU Virtusa Corp.
Processing:VRTV Veritiv Corp.
Processing:VRTV Veritiv Corp.
Processing:VSEC VSE Corp.
Processing:VSEC VSE Corp.
Processing:VSH Vishay Intertechnology, Inc.
Processing:VSH Vishay Intert

,,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,receivablesTurnover,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,...,5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,% return on investment
symbol,name,,,,,,,,,,,,,,,,,,,,,
AAN,"Aaron's, Inc.",1.53150,1.95610,0.78855,12.37860,27.77570,59.12785,19.92670,0.0,0.0,0.0,...,0.09225,0.06350,0.08230,0.00000,0.10410,0.12870,-0.38070,0.00000,0.08135,97.601226
AAON,"AAON, Inc.",3.41760,3.41760,1.63730,16.86505,6.73510,20.69945,10.96230,0.0,0.0,0.0,...,0.00000,0.08890,0.02775,-0.02075,0.06795,0.13045,-0.98780,0.00000,-0.02605,369.576073
AAT,"American Assets Trust, Inc.",0.00000,0.00000,7.06720,261.50000,3.15505,18.73490,17.21230,0.0,0.0,0.0,...,0.00000,0.00000,-0.22290,0.00000,0.07845,0.00000,0.07855,0.00000,0.02340,122.034670
AAWW,"Atlas Air Worldwide Holdings, Inc.",1.19360,1.24460,0.91035,10.26345,18.45670,5.75450,4.40720,0.0,0.0,0.0,...,0.00000,0.00000,0.07730,0.00000,0.09980,0.13710,-0.14835,0.00000,0.01570,14.055966
AAXN,"Axon Enterprise, Inc.",2.40520,2.46675,2.99445,0.00000,6.24490,0.00000,214.55715,0.0,0.0,0.0,...,0.00000,0.00000,-0.10185,0.15055,0.02425,0.02140,0.00000,0.05945,0.00815,600.346411
ABCB,Ameris Bancorp,0.70355,0.71705,2.21820,0.00000,2.30185,5.57455,4.78655,0.0,0.0,0.0,...,-0.20995,-0.34500,1.43265,0.00000,0.11660,-0.19010,0.12465,0.00000,0.14120,857.817274
ABG,"Asbury Automotive Group, Inc.",1.80070,1.87375,0.13560,21.55670,41.62940,1.77900,32.23090,0.0,0.0,0.0,...,0.00000,-1.00000,0.12990,-0.07795,-0.04520,0.11810,-0.10120,0.00000,-0.05555,168.482291
ABM,"ABM Industries, Inc.",1.49475,13.45235,0.30525,17.88545,6.62630,8.54645,7.31090,0.0,0.0,0.0,...,0.05270,0.04865,-0.01630,0.00000,-0.00835,0.05630,-0.21775,0.00000,-0.08370,123.398857
ACAD,"ACADIA Pharmaceuticals, Inc.",2.87115,2.87115,4.50650,1.53845,0.00000,0.00000,-4.04300,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,-0.22955,0.30350,-0.72675,-0.38615,-0.25075,5840.360646


In [6]:
dataframe2 = dataframe2.drop(columns=['priceCashFlowRatio', 'priceEarningsToGrowthRatio', 'priceSalesRatio', 'dividendYield', 'priceFairValue', 'operatingProfitMargin'])

In [7]:
# filtering the completed dataframe by creating a new dataframe that the does not have the rows associated with 0 in the % return on investment column.
dataframe2 = dataframe2[dataframe2['% return on investment'] != 0]

In [9]:
dataframe2.to_csv('Russell.csv')